In [1]:
# for Python 2: use print only as a function
# from __future__ import print_function

In [2]:
import ast

import pandas as pd
import numpy as np
from collections import defaultdict
import collections

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from sklearn.pipeline import Pipeline

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from sklearn.preprocessing import MultiLabelBinarizer 
from sklearn.multiclass import OneVsRestClassifier 
from sklearn.multioutput import MultiOutputClassifier                  # included from scikit-learn version 0.18.1 and onwards
from sklearn.ensemble import RandomForestClassifier

In [3]:
path = 'C:\Users\Pushpendra\Desktop\po\datascience_datasets.csv'

In [4]:
d_set = pd.read_csv(path,header=None,names=['tag','title'],dtype=object)

In [5]:
d_set.head()

,tag,title
0,"['r', 'regression', 'categorical-data']",Dummy coding a column in R with multiple levels
1,"['r', 'sequential-pattern-mining', 'traminer']",Sequence pattern mining on continuous dataset
2,"['machine-learning', 'classification', 'evalua...",How to improve an existing (trained) classifier?
3,"['machine-learning', 'data-mining', 'apache-sp...",Machine Learning model to find items that are ...
4,['xgboost'],Extracting information from DMatrix in xgboost...


In [6]:
y_labels = []                                                                  # contains values of d_set['tag']
for i in d_set['tag']:  
    y_labels.append(ast.literal_eval(i))                                       # to remove unicodeed string 
y_labels = [j for i in y_labels for j in i] 
y_labels = list(set(y_labels))

# for i in range(len(y_labels)):
#     if (y_labels[i].find('-') > -1):
#         y_labels[i] = y_labels[i].replace('-','')

In [7]:
import re
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

class lemmatokenizer(object):
    def __init__(self):
        self.stemmer = SnowballStemmer('english')
        self.token_pattern = r"(?u)\b\w\w+\b"       
#         self.wnl = WordNetLemmatizer()
    def __call__(self,doc):                                                     # here, doc is one string sentence
        token_pattern = re.compile(self.token_pattern)
        return [self.stemmer.stem(t) for t in token_pattern.findall(doc)]       # return lambda doc: token_pattern.findall(doc) 
#         return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [8]:
vect_title = CountVectorizer(max_df=0.5,min_df=5,stop_words='english',tokenizer=lemmatokenizer(),ngram_range=(1,3))

In [9]:
tfidf_vect_title = TfidfVectorizer(smooth_idf=False,max_df=0.5,min_df=5,stop_words='english',tokenizer=lemmatokenizer(),ngram_range=(1,3))

# To make it clear what actually we are doing...

### tag                           
['r', 'machine-learning', 'ai'] 
   
### Labelencoder() 
[32, 324, 17]              
   
### MultiLabelBinarizer()
[0,0,0,0,........1,0,0,1,1,0,...]


In [10]:
le = preprocessing.LabelEncoder()  
le.fit(y_labels) 
d_set['label_num'] = pd.Series([le.transform(ast.literal_eval(i)) for i in d_set['tag']])
d_set.head()

,tag,title,label_num
0,"['r', 'regression', 'categorical-data']",Dummy coding a column in R with multiple levels,"[208, 216, 34]"
1,"['r', 'sequential-pattern-mining', 'traminer']",Sequence pattern mining on continuous dataset,"[208, 236, 267]"
2,"['machine-learning', 'classification', 'evalua...",How to improve an existing (trained) classifier?,"[150, 36, 83]"
3,"['machine-learning', 'data-mining', 'apache-sp...",Machine Learning model to find items that are ...,"[150, 59, 13, 113]"
4,['xgboost'],Extracting information from DMatrix in xgboost...,[281]


In [11]:
new_y_labels = d_set['label_num'].values.tolist()

# print (new_y_labels)

In [12]:
mlb = MultiLabelBinarizer() 
mlb.fit(new_y_labels)

# mlb.transform(new_y_labels).shape

MultiLabelBinarizer(classes=None, sparse_output=False)

In [13]:
y_tag_dtm = mlb.transform(new_y_labels) 

# print (type(y_tag_dtm))
# y_tag_dtm = pd.Series(y_tag_dtm) 

y_tag_dtm.shape

(3548, 282)

In [14]:
X_labels = d_set['title'].values.tolist()

# print (X_labels)

In [15]:
vect_title.fit(X_labels)
X_title_dtm = vect_title.transform(X_labels)

# vect_title.get_feature_names()
# vect_title.get_params
# print (X_title_dtm.toarray())

X_title_dtm

<3548x922 sparse matrix of type '<type 'numpy.int64'>'
	with 18043 stored elements in Compressed Sparse Row format>

# Feature Transformation
## implementation PCA :
I don't know why but for some reason it's not doing its job for MultiLabel output...it's throwing error 
during the training of classifier (both knn_clf and rf_clf) . 

In [16]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100).fit(X_title_dtm.toarray())
pca_samples = pca.transform(X_title_dtm.toarray())

pca_df = pd.DataFrame(np.round(pca_samples,4))

print (pca_df.head())

       0       1       2       3       4       5       6       7       8   \
0 -0.1665 -0.1138 -0.1273 -0.1345 -0.1034  0.0072 -0.0181  0.0605 -0.0463   
1 -0.1684 -0.1009 -0.1551 -0.1138 -0.1179  0.0110 -0.1246  0.0034 -0.0352   
2 -0.1543 -0.1186 -0.1223 -0.0421 -0.0548 -0.0232  0.0761 -0.2428  0.4300   
3 -0.4502  1.6132  0.3952  0.8012  0.7419 -0.3519 -0.3020 -0.0869 -0.3281   
4 -0.2185 -0.1189 -0.1356 -0.0682 -0.1036 -0.0390 -0.0825  0.0239 -0.0624   

       9    ...        90      91      92      93      94      95      96  \
0 -0.0123   ...    0.0582  0.0038  0.0260 -0.2615 -0.0631  0.3202 -0.0066   
1 -0.0273   ...    0.0153  0.1127 -0.2583 -0.1806  0.0200 -0.1053  0.0678   
2 -0.3340   ...    0.0054  0.0076  0.0501  0.0410  0.0152 -0.0531  0.0049   
3 -0.2772   ...    0.0831 -0.1005  0.1559  0.1070  0.2640  0.0794  0.2680   
4 -0.0411   ...    0.1493 -0.0826  0.1375  0.3090 -0.0828  0.0074  0.1183   

       97      98      99  
0  0.4048  0.0513 -0.0091  
1  0.0162  0.1068 

In [17]:
new_df = pd.DataFrame(X_title_dtm.toarray(),columns=vect_title.get_feature_names())

# new_df.head()
# new_df.ix[1].to_dict().values()

new_df.shape

(3548, 922)

In [18]:
d = collections.Counter(vect_title.get_feature_names())
# print (d['ai']) 

In [19]:
new_df['target_list'] = [i for i in y_tag_dtm] 

In [20]:
# new_df.columns[:100]  
# new_df.ix[0]

In [21]:
# new_df.head()  

In [22]:
tfidf_vect_title.fit(X_labels)
X_title_dtm_tfidf = tfidf_vect_title.transform(X_labels)

X_title_dtm_tfidf

<3548x922 sparse matrix of type '<type 'numpy.float64'>'
	with 18043 stored elements in Compressed Sparse Row format>

In [23]:
new_df_of_tfidf = pd.DataFrame(X_title_dtm_tfidf.toarray(),columns=tfidf_vect_title.get_feature_names()) 

In [24]:
new_df_of_tfidf['target_list'] = [i for i in y_tag_dtm] 

In [25]:
y = new_df_of_tfidf['target_list'] 
X = new_df_of_tfidf.drop('target_list',axis=1)  

In [26]:
# y = new_df['target_list']
# X = new_df.drop('target_list',axis=1)

# # X = X.ix[:]                                           # it will return each feature row wise to X
# # X = X.values

# print (type(X))

In [27]:
X = np.array(X.values.tolist())                           # it will convert list to numpy ndarray
y = np.array(y.values.tolist())

In [28]:
# print (X[0]) 

In [29]:
pca_X = PCA(n_components=200).fit_transform(X)  
pca_X = np.round(pca_X,4)

pca_y = PCA(n_components=50).fit_transform(y)  
pca_y = np.round(pca_y,4)

In [30]:
print (pca_y) 

[[-0.392  -0.3133  0.7996 ..., -0.056   0.1193  0.0281]
 [-0.3741 -0.3108  0.7694 ..., -0.0126  0.0037 -0.0357]
 [ 0.8004 -0.1387 -0.1726 ..., -0.0014  0.0079  0.0908]
 ..., 
 [-0.2878 -0.1038 -0.0992 ..., -0.0085 -0.0222  0.0042]
 [-0.2894 -0.0968 -0.1342 ..., -0.012  -0.0047 -0.0144]
 [-0.3434 -0.0898 -0.1336 ..., -0.0271  0.0426 -0.1531]]


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)   

In [32]:
# X_train, X_test, y_train, y_test = train_test_split(pca_X, pca_y, test_size=0.2, random_state=1)   

In [54]:
# clf = Pipeline([('classifier',OneVsRestClassifier(SVC(probability=True,random_state=0)))])  # just to for Pipeline example

knn_clf = KNeighborsClassifier(n_neighbors=5)
# mnb_clf = MultinomialNB()                                                                   # not working for MultiLabelinput

# time_pass_y = np.random.randint(2,size=(2838,1))                                            # produce ndarray of size 2838 X 1

knn_clf.fit(X_train, y_train)
# mnb_clf.fit(X_train, y_train) 

knn_pred = knn_clf.predict(X_test)  
# mnb_pred = mnb_clf.predict(X_test)

In [55]:
svc_clf = OneVsRestClassifier(SVC(probability=True,random_state=0))
svc_clf.fit(X_train, y_train)
svc_pred = svc_clf.predict(X_test)  

In [34]:
knn_clf.score(X_test, y_test) 

0.033802816901408447

In [56]:
svc_clf.score(X_test, y_test) 

0.0

In [57]:
from sklearn import metrics

knn_report = metrics.classification_report(y_test[:], knn_pred[:]) 
knn_f1_score = metrics.f1_score(y_test[:], knn_pred[:], average='samples') 
knn_precision_recall_fscore = metrics.precision_recall_fscore_support(y_test, knn_pred, average='samples')  # on full data-set
knn_avg_precision_score = metrics.average_precision_score(y_test, knn_pred, average='samples')
knn_roc_auc_score = metrics.roc_auc_score(y_test, knn_pred, average='samples')

# mnb_report = metrics.classification_report(y_test[:100], mnb_pred[:100])  #throwing error mnb_clf can't work on multilabel O/P

In [59]:
svc_report = metrics.classification_report(y_test[:], svc_pred[:]) 
svc_f1_score = metrics.f1_score(y_test[:], svc_pred[:], average='samples') 
svc_precision_recall_fscore = metrics.precision_recall_fscore_support(y_test, svc_pred, average='samples')  # on full data-set
svc_avg_precision_score = metrics.average_precision_score(y_test, svc_pred, average='samples')
svc_roc_auc_score = metrics.roc_auc_score(y_test, svc_pred, average='samples')

In [60]:
metrics.accuracy_score(y_true=y_test[:], y_pred=svc_pred[:])          # I think it's same as calculating hamming_score

0.0

In [61]:
print "For svc_clf (LinearSVC) : "
print "precision, recall, fbeta_score, support : ",svc_precision_recall_fscore
print "f1_score : ",svc_f1_score
print "avg. precision_score : ",svc_avg_precision_score 
print "roc_auc_score : ",svc_roc_auc_score

For svc_clf (LinearSVC) : 
precision, recall, fbeta_score, support :  (0.0, 0.0, 0.0, None)
f1_score :  0.0
avg. precision_score :  0.504609929078
roc_auc_score :  0.5


In [58]:
metrics.accuracy_score(y_true=y_test[:], y_pred=knn_pred[:])          # I think it's same as calculating hamming_score

0.033802816901408447

In [37]:
# print (knn_report)                                   # its type is str

print "For knn_clf (KNearestNeighbours) : "
print "precision, recall, fbeta_score, support : ",knn_precision_recall_fscore
print "f1_score : ",knn_f1_score
print "avg. precision_score : ",knn_avg_precision_score 
print "roc_auc_score : ",knn_roc_auc_score

For knn_clf (KNearestNeighbours) : 
precision, recall, fbeta_score, support :  (0.28286384976525825, 0.13798122065727697, 0.17197406662195394, None)
f1_score :  0.171974066622
avg. precision_score :  0.401754569973
roc_auc_score :  0.568224767594


In [38]:
# def does_test_tag_match(d, list_of_tags):      # no need for this function

In [39]:
test = ["how to use policy iteration in ml ?"]
# test = ["what is lstm ?"] 

# test_dtm = vect_title.transform(test)                                           # without tfidf
test_dtm = tfidf_vect_title.transform(test)                                       # with tfidf

# print (test_dtm.toarray()[0])
status = False
for i in test_dtm.toarray()[0]:
    if (i!=0):
        status = True
        break

ans = knn_clf.predict(test_dtm.toarray())
ans = mlb.inverse_transform(ans)

if (len(ans[0])==0 or status==False):
    print ("sorry, we can't predict your category!!!")
else:
    ans = le.inverse_transform(ans)
    print (ans)
    
    
# mlb.transform([[224,0,100]]) 
# ans 
# test_dtm.toarray()

[['machine-learning']]


In [40]:
forest = RandomForestClassifier(n_estimators=10, random_state=0)
rf_clf = MultiOutputClassifier(forest, n_jobs=-1)
rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)

In [41]:
rf_clf 

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False),
           n_jobs=-1)

In [42]:
metrics.accuracy_score(y_true=y_test[:100], y_pred=rf_pred[:100])          # I think it's same as calculating hamming_score

0.059999999999999998

In [43]:
rf_clf.score(X_test, y_test)

0.040845070422535212

In [44]:
# rf_clf.predict_log_proba(X_test)                                               # no such function in MultiOutputClassifier()

In [45]:
# print (knn_clf.predict_proba(X_train))                                         # running smooth

# print (rf_clf.predict_proba(X_train[:5]))                                      # showing error

In [46]:
rf_report = metrics.classification_report(y_test[:], rf_pred[:])
rf_f1_score = metrics.f1_score(y_test, rf_pred, average='samples')  
rf_precision_recall_fscore = metrics.precision_recall_fscore_support(y_test, rf_pred, average='samples')  # on full data-set
rf_avg_precision_score = metrics.average_precision_score(y_test, rf_pred, average='samples')
rf_roc_auc_score = metrics.roc_auc_score(y_test, rf_pred, average='samples') 

In [47]:
# print (rf_report) 

print "For rf_clf (RandomForest) : "
print "precision, recall, fbeta_score, support : ",rf_precision_recall_fscore
print "f1_score : ",rf_f1_score  
print "avg. precision_score : ",rf_avg_precision_score 
print "roc_auc_score : ",rf_roc_auc_score

For rf_clf (RandomForest) : 
precision, recall, fbeta_score, support :  (0.332018779342723, 0.1898826291079812, 0.22276995305164318, None)
f1_score :  0.222769953052
avg. precision_score :  0.468282639097
roc_auc_score :  0.594238774341


In [48]:
# test = ["what is reinforcement learning ?"] 
test = ["what is ai,lstm and data visualization ?"] 

# test_dtm = vect_title.transform(test)                                            # without tfidf
test_dtm = tfidf_vect_title.transform(test)                                        # with tfidf

status = False
for i in test_dtm.toarray()[0]:
    if (i!=0):
        status = True
        break

ans = rf_clf.predict(test_dtm.toarray())
ans = mlb.inverse_transform(ans)
if (len(ans[0])==0 or status==False):
    print ("sorry, we can't predict your category!!!")
else:
    ans = le.inverse_transform(ans)
    print (ans)
    
# mlb.transform([[224,0,100]]) 
# ans 
# test_dtm.toarray()

[['neuralnetwork' 'visualization']]


In [49]:
from sklearn.externals import joblib
joblib.dump(rf_clf, 'datascience_classifier.pkl')
# new_clf = joblib.load('classifier.pkl')

['datascience_classifier.pkl']

In [50]:
new_pkl_clf = joblib.load('datascience_classifier.pkl')

In [51]:
new_pkl_clf

MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=0,
            verbose=0, warm_start=False),
           n_jobs=-1)

In [52]:
test = ["How to use policy iteration in ml ?"] 

# test_dtm = vect_title.transform(test)                                           # without tfidf
test_dtm = tfidf_vect_title.transform(test)                                       # with tfidf       

status = False
for i in test_dtm.toarray()[0]:
    if (i!=0):
        status = True
        break
        
ans = new_pkl_clf.predict(test_dtm.toarray())
ans = mlb.inverse_transform(ans)
if (len(ans[0])==0 or status==False):
    print ([["sorry, we can't predict your category!!!"]]) 
else:
    ans = le.inverse_transform(ans)
    print (ans)

[["sorry, we can't predict your category!!!"]]
